<a href="https://colab.research.google.com/github/shahpriyanka26/shahpriyanka26/blob/main/Copy_of_BioBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd drive/MyDrive/TopicModeling_TextSum/NER

[Errno 2] No such file or directory: 'drive/MyDrive/TopicModeling_TextSum/NER'
/content/drive/MyDrive/TopicModeling_TextSum/NER


In [ ]:
import pandas as pd

In [ ]:
# !git clone https://github.com/dmis-lab/biobert-pytorch

In [ ]:
%cd biobert-pytorch

/content/drive/MyDrive/TopicModeling_TextSum/NER/biobert-pytorch


In [ ]:
# Install huggingface transformers
!pip install transformers==3.0.0
!pip install seqeval

     |████████████████████████████████| 754 kB 4.2 MB/s 
     |████████████████████████████████| 880 kB 47.5 MB/s 
     |████████████████████████████████| 1.2 MB 36.5 MB/s 
     |████████████████████████████████| 3.0 MB 32.4 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=36d16057ced8b5a3102d71a8613b0647ccea70609a55f4e9c95bcd64b6ead96e
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
     |████████████████████████████████| 43 kB 1.4 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=666bb0466bf260053679da248ad551f1407e6450680be713de1601c600be16c2
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval


In [ ]:
# !tar -xvf "datasets.tar"

In [ ]:
# Pre-process NER datasets
%cd named-entity-recognition
!bash preprocess.sh

[Errno 2] No such file or directory: 'named-entity-recognition'
/content/drive/MyDrive/TopicModeling_TextSum/NER/biobert-pytorch/named-entity-recognition
*****  NCBI-disease  Preprocessing Start *****
Replacing Done
Downloading: 100% 433/433 [00:00<00:00, 284kB/s]
Downloading: 100% 213k/213k [00:00<00:00, 294kB/s]
*****  NCBI-disease  Preprocessing Done *****
*****  BC5CDR-disease  Preprocessing Start *****
Replacing Done
*****  BC5CDR-disease  Preprocessing Done *****
*****  BC5CDR-chem  Preprocessing Start *****
Replacing Done
*****  BC5CDR-chem  Preprocessing Done *****
*****  BC4CHEMD  Preprocessing Start *****
Replacing Done
*****  BC4CHEMD  Preprocessing Done *****
*****  JNLPBA  Preprocessing Start *****
Replacing Done
*****  JNLPBA  Preprocessing Done *****
*****  BC2GM  Preprocessing Start *****
Replacing Done
*****  BC2GM  Preprocessing Done *****
*****  linnaeus  Preprocessing Start *****
Replacing Done
*****  linnaeus  Preprocessing Done *****
*****  s800  Preprocessing Sta

##Creating Script for making Prediction

In [ ]:
from transformers import (AutoTokenizer, 
                          AutoModelForTokenClassification, 
                          AutoConfig, 
                          Trainer,
                          EvalPrediction,
                          TrainingArguments,
                          HfArgumentParser
)
from transformers import pipeline
from utils_ner import NerDataset, Split, get_labels
from typing import Dict, Tuple, List
import numpy as np
from run_ner import ModelArguments, DataTrainingArguments
import sys
from torch import nn
from seqeval.metrics import f1_score, precision_score, recall_score

In [ ]:
def align_predictions(predictions: np.ndarray, label_ids: np.ndarray) -> Tuple[List[int], List[int]]:
      preds = np.argmax(predictions, axis=2)

      batch_size, seq_len = preds.shape

      out_label_list = [[] for _ in range(batch_size)]
      preds_list = [[] for _ in range(batch_size)]
      
      for i in range(batch_size):
          for j in range(seq_len):
              if label_ids[i, j] != nn.CrossEntropyLoss().ignore_index:
                  out_label_list[i].append(label_map[label_ids[i][j]])
                  preds_list[i].append(label_map[preds[i][j]])

      return preds_list, out_label_list

def compute_metrics(p: EvalPrediction) -> Dict:
    preds_list, out_label_list = align_predictions(p.predictions, p.label_ids)
    
    return {
        "precision": precision_score(out_label_list, preds_list),
        "recall": recall_score(out_label_list, preds_list),
        "f1": f1_score(out_label_list, preds_list),
    }


dir = "/content/drive/MyDrive/TopicModeling_TextSum/NER/biobert-pytorch"
labels_path = dir + "/datasets/NER/NCBI-disease/labels.txt"
labels = get_labels(labels_path)
label_map: Dict[int, str] = {i: label for i, label in enumerate(labels)}
num_labels = len(labels)

config = AutoConfig.from_pretrained(dir + "/named-entity-recognition/output/NCBI-disease",
        num_labels= num_labels,
        id2label= label_map,
        label2id= {label: i for i, label in enumerate(labels)},
        cache_dir= dir + "/testing",
)

tokenizer = AutoTokenizer.from_pretrained(dir + "/named-entity-recognition/output/NCBI-disease",
    cache_dir= dir + "/testing",
    use_fast= False,
)

model = AutoModelForTokenClassification.from_pretrained(
    dir + "/named-entity-recognition/output/NCBI-disease",
    from_tf= bool(".ckpt" in dir + "/named-entity-recognition/output/NCBI-disease"),
    config= config,
    cache_dir= dir + "/testing",
)

test_dataset = NerDataset(dir + "/datasets/NER/NCBI-disease",
          tokenizer= tokenizer,
          labels= labels,
          model_type= config.model_type,
          max_seq_length= 192,
          overwrite_cache= True,
          mode= Split.test,
)
train_dataset = NerDataset(dir + "/datasets/NER/NCBI-disease",
          tokenizer= tokenizer,
          labels= labels,
          model_type= config.model_type,
          max_seq_length= 192,
          overwrite_cache= True,
          mode= Split.train,
)
eval_dataset = NerDataset(dir + "/datasets/NER/NCBI-disease",
          tokenizer= tokenizer,
          labels= labels,
          model_type= config.model_type,
          max_seq_length= 192,
          overwrite_cache= True,
          mode= Split.dev,
)

ModelArgs = ModelArguments(
            model_name_or_path= dir + "/named-entity-recognition/output/NCBI-disease",
            cache_dir= dir + "/testing"
)

DataTrainArgs = DataTrainingArguments(
                data_dir= dir + "/datasets/NER/NCBI-disease",
                labels= dir + "/datasets/NER/NCBI-disease/labels.txt",
                overwrite_cache = True
)
TrainArgs = TrainingArguments(output_dir= dir + "/testing")
# parser = HfArgumentParser((ModelArgs, DataTrainArgs, TrainArgs))
# # model_args, data_args, training_args = parser.parse_json_file(json_file=os.path.abspath(sys.argv[1]))
# model_args, data_args, training_args = parser.parse_args_into_dataclasses()

TrainArgs = TrainingArguments(output_dir= dir + "/testing")
trainer = Trainer(
    model= model,
    args= TrainArgs,
    train_dataset= train_dataset,
    eval_dataset= eval_dataset,
    compute_metrics= compute_metrics,
)

In [ ]:
predictions, label_ids, metrics = trainer.predict(test_dataset)
preds_list, _ = align_predictions(predictions, label_ids)

Prediction:   0%|          | 0/118 [00:00<?, ?it/s]

In [ ]:
metrics

{'eval_f1': 0.870096790626592,
 'eval_loss': 0.04990868259396925,
 'eval_precision': 0.8514456630109671,
 'eval_recall': 0.8895833333333333}

In [ ]:
def split(m_str):
  m_list = m_str.split()
  if len(m_list)>1:
    return m_list[0]
  else:
    return ''

In [ ]:
#read test file
textfile = open(dir + "/datasets/NER/NCBI-disease/test.txt", "r")
lines = textfile.readlines()
textfile.close()
lines_f = list(map(lambda x: split(x), lines))

In [ ]:
#save test file without labels
textfile = open(dir + "/datasets/NER/NCBI-disease/dummy_test.txt", "w+")
for element in lines_f:
    textfile.write(element + "\n")
textfile.close()

In [ ]:
dummy_test_dataset = NerDataset(dir + "/datasets/NER/NCBI-disease",
        tokenizer= tokenizer,
        labels= labels,
        model_type= config.model_type,
        max_seq_length= 192,
        overwrite_cache= True,
        mode= Split.dum_test,
)

In [ ]:
dummy_test_preds, dummy_test_label_ids, dummmy_metrics = trainer.predict(dummy_test_dataset)
dummy_test_preds_list, _ = align_predictions(dummy_test_preds, dummy_test_label_ids)

Prediction:   0%|          | 0/118 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
dummy_test_preds_list_flat = [item for sublist in dummy_test_preds_list for item in sublist]
preds_list_flat = [item for sublist in preds_list for item in sublist]

In [ ]:
compare_dum_test = {'test_with_labels': preds_list_flat,
                    'test_wo_labels': dummy_test_preds_list_flat
                    }

In [ ]:
compare_df = pd.DataFrame(compare_dum_test)
compare_df.to_csv(dir + "/named-entity-recognition/output/NCBI-disease/compare_dum_test.csv")

In [ ]:
iqvia_data = pd.read_csv(dir + "/datasets/NER/NCBI-disease/sub_all_merged.csv")
iqvia_data.head()

,Unnamed: 0,id,type,merged,en_core_sci_scibert,CUI_umls,token_standard_name,LDA_topic,Cluster_labels
0,0,31761661,bipolar,"A Population-Based, Nationwide Longitudinal St...","Bipolar Disorder, Incident, Dementia, Taiwan, ...","C1551358, C0035647, C0205217, C0525045, C09476...","incident, Risk, Increased, Mood Disorders, Sci...",6,6
1,1,30827282,bipolar,Understanding the association between advanced...,"association, advanced paternal age, schizophre...","C0035647, C0001792, C0036341, C0947630, C02052...","Risk, Elderly (population group), Schizophreni...",6,1
2,2,30714166,bipolar,Identity in bipolar disorder: Self-worth and a...,"Identity, bipolar disorder, Self-worth, achiev...","C3714552, C0439064, C0678966, C0005586, C00180...","Weakness, Numerous, personality psychology, Bi...",6,7
3,3,31840225,bipolar,Three-year longitudinal cognitive functioning ...,"Three-year, longitudinal, cognitive functionin...","C0349208, C0027902, C0220825, C0205127, C00055...","manic episode, Neuropsychological Tests, Evalu...",6,7
4,4,31704596,bipolar,Relationship of elevated neural cell adhesion ...,"Relationship, elevated, neural cell adhesion m...","C1707520, C0026777, C0005586, C1511726, C00278...","Correlation, Multivariate Analysis, Bipolar Di...",8,3


In [ ]:
iqvia_data['en_core_sci_scibert'][0].replace(' ', '').split(',')
#save iqvia data in the format which can be given to model for prediction
iqvia_tf = open(dir + "/datasets/NER/NCBI-disease/iqvia_data.txt", "w+")
for element in lines_f:
   iqvia_tf.write(element + "\n")
iqvia_tf.close()

['BipolarDisorder',
 'Incident',
 'Dementia',
 'Taiwan',
 'Affectivedisorders',
 'associatedwith',
 'increased',
 'risk',
 'dementia',
 'studies',
 'association',
 'majordepressivedisorder',
 'depressiveepisodes',
 'bipolardisorder',
 'increased',
 'dementia',
 'risk',
 'association',
 'manic/mixedepisodes',
 'bipolardisorder',
 'increased',
 'dementia',
 'risk',
 'unclear']

In [ ]:
!export SAVE_DIR=./output
!export DATA_DIR=../datasets/NER

!export MAX_LENGTH=192
!export BATCH_SIZE=32
!export NUM_EPOCHS=30
!export SAVE_STEPS=1000
!export ENTITY=NCBI-disease
!export SEED=1

!python run_ner.py \
    --data_dir ../datasets/NER/NCBI-disease/ \
    --labels ../datasets/NER/NCBI-disease/labels.txt \
    --model_name_or_path dmis-lab/biobert-base-cased-v1.1 \
    --output_dir ./output/NCBI-disease \
    --max_seq_length 192 \
    --num_train_epochs 3 \
    --per_device_train_batch_size 32 \
    --save_steps 1000\
    --seed 1\
    --do_train \
    --do_eval \
    --do_predict \
    --overwrite_output_dir

In [ ]:
pre_trained_path = "/content/drive/MyDrive/TopicModeling_TextSum/NER/biobert-pytorch/named-entity-recognition/output/NCBI-disease"

tokenizer = AutoTokenizer.from_pretrained(pre_trained_path, model_max_length=512)
# model = AutoModelForTokenClassification.from_pretrained(pre_trained_path)
# nlp=pipeline(task='ner',model=model,tokenizer=tokenizer,grouped_entities=True,)

In [ ]:
text="Electroconvulsive Therapy in Children and Adol."
nlp(text)

[]

In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/TopicModeling_TextSum/sdummy.csv")
df.head()

,id,text
0,34427054,A rare case report of teen-onset pica in a fem...
1,34424874,Electroconvulsive Therapy in Children and Adol...
2,34423843,Dopamine D1R Receptor Stimulation as a Mechani...
3,34421684,Advancing Autism Research From Mice to Marmose...
4,34421674,Altered Spatial Organization of Dynamic Functi...


In [ ]:
texts = df['text']
outputs = list(map(lambda x: nlp(x), texts[:10]))

In [ ]:
outputs[1]

[{'entity_group': 'B-bio', 'score': 0.9997295141220093, 'word': 'psychiatric'},
 {'entity_group': 'I-bio', 'score': 0.9975121021270752, 'word': 'diseases'},
 {'entity_group': 'B-bio', 'score': 0.9976486563682556, 'word': 'psychiatric'},
 {'entity_group': 'I-bio', 'score': 0.9851688146591187, 'word': 'diseases'},
 {'entity_group': 'B-bio', 'score': 0.9999804496765137, 'word': 'mood'},
 {'entity_group': 'I-bio', 'score': 0.9999707937240601, 'word': 'disorders'},
 {'entity_group': 'B-bio', 'score': 0.9999865293502808, 'word': 'cat'},
 {'entity_group': 'I-bio', 'score': 0.9998069107532501, 'word': '##atonia'},
 {'entity_group': 'B-bio', 'score': 0.9043158888816833, 'word': 'schi'},
 {'entity_group': 'I-bio', 'score': 0.9999613165855408, 'word': '##zophrenia'},
 {'entity_group': 'B-bio',
  'score': 0.9999850392341614,
  'word': 'intellectual'},
 {'entity_group': 'I-bio', 'score': 0.9999822974205017, 'word': 'disability'},
 {'entity_group': 'B-bio', 'score': 0.9999767541885376, 'word': 'cogn

In [ ]:
df_result = pd.DataFrame(outputs)
df_result.to_csv("sdummy_ner_result.csv")

In [ ]:
tokens = list(map(lambda x: tokenizer.tokenize(x), texts))

In [ ]:
df_tokens = pd.DataFrame(tokens)
df_tokens.to_csv("sdummy_tokens.csv")

In [ ]:
df_tokens.head()

,0,1,2,3,4,5,6,7,8,9,...,1098,1099,1100,1101,1102,1103,1104,1105,1106,1107
0,a,rare,case,report,of,teen,-,onset,p,##ica,...,None,None,None,None,None,None,None,None,None,None
1,electro,##con,##vu,##lsive,therapy,in,children,and,adolescent,##s,...,None,None,None,None,None,None,None,None,None,None
2,do,##pa,##mine,d,##1,##r,receptor,stimulation,as,a,...,None,None,None,None,None,None,None,None,None,None
3,advancing,autism,research,from,mice,to,ma,##rm,##ose,##ts,...,None,None,None,None,None,None,None,None,None,None
4,altered,spatial,organization,of,dynamic,functional,network,associates,with,def,...,None,None,None,None,None,None,None,None,None,None


In [ ]:
def CountFreq(my_list):
     
   # Creating an empty dictionary
   count = {}
   for i in my_list:
    count[i] = count.get(i, 0) + 1
   return count

In [ ]:
flat_tokens = [token for sublist in tokens for token in sublist]

In [ ]:
unique_tokens = list(set(flat_tokens))

In [ ]:
token_freq = CountFreq(flat_tokens)

In [ ]:
def join_tokens(t1, t2):
  t = ''.join([t1, t2])
  return t

In [ ]:
''.join(['d', 'g'])

'dg'

In [ ]:
def find_sub_tokens(token_list):
  for i, token in enumerate(token_list):
    if token.find('##') == 0:
      sub_tokens.append(token_list[i-1])

1

In [ ]:
for i, t in enumerate([3, 4, 5]):
  print(i)

0
1
2
